In [1]:
import yaml
import logging
import os
import warnings
warnings.filterwarnings("ignore")
os.chdir("workdir")


# def load_yaml(file_path: str):
#   """
#   Load a rule from a YAML file.
#   """
#   with open(file_path) as f:
#     data = yaml.load(f, Loader=yaml.FullLoader)

#   logging.info(f"Se cargó la configuración en: {file_path}")
#   return data


# config = load_yaml("./local/input/config.yaml")
# logging.info(f"config file load: {config}")

config = {
  "ticker": "NU",
  "financial_weights": {"income": {"growth": 0.5, "peers": 0.5}},
  "peers": {"custom": None, "n_competitors": 5},
  "multiples_weights": {"pe_ratio": 0.25, "ps_ratio": 0.25, "pgp_ratio": 0.25, "pfcf_ratio": 0.25}
}

ticker = config["ticker"]
weights = config["financial_weights"]
peers_cfg = config["peers"]
multiples_weights = config["multiples_weights"]

# --------------------------------
from main import execute_process

response = execute_process(
  ticker=ticker,
  financial_weights=weights,
  peers=peers_cfg,
  multiples_weights=multiples_weights
)

INFO:root:Tipo de resultados: income
INFO:root:KPI's analizar: ['Revenue', 'Gross Profit', 'Operating Income', 'Net Income']

INFO:root:Request successful to url: https://stockanalysis.com/stocks/nu/financials/
INFO:root:No se encontro el kpi Gross Profit... ajustado al Revenue
INFO:root:earnings date: ['2024-09-30' '2023-12-31' '2022-12-31' '2021-12-31' '2020-12-31'
 '2019-12-31']
INFO:root:Vamos a revisar que la empresa NU se encuentre creciendo
INFO:root:========================================
INFO:root:Analizando crecimiento de la compañía...
INFO:root:kpi: Revenue analizado. Proporcion de periodos de crecimiento: 1.0
INFO:root:kpi: Operating Income analizado. Proporcion de periodos de crecimiento: 0.8
INFO:root:kpi: Net Income analizado. Proporcion de periodos de crecimiento: 0.6
INFO:root:kpi: Gross Profit analizado. Proporcion de periodos de crecimiento: 1.0
INFO:root:========================================
INFO:root:Score de crecimiento: 8.5
INFO:root:Calculando los margenes 

In [2]:
response

{'financials': {'income': {'income_complete':                             nding Sep 30, 2024 Dec 31, 2023 Dec 31, 2022  \
   0    Interest and Dividend Income         9089         6440         3555   
   1          Total Interest Expense         2514         2037         1548   
   2             Net Interest Income         6575         4403         2007   
   3            Commissions and Fees         1844         1589         1237   
   4      Revenue Before Loan Losses         8419         5992         3244   
   5       Provision for Loan Losses         2957         2285         1405   
   6                         Revenue         5461         3707         1839   
   7            Revenue Growth (YoY)       73.49%      101.52%      116.39%   
   8    Salaries & Employee Benefits       350.21       255.61       286.45   
   9       Cost of Services Provided         1726         1446         1180   
   10       Other Operating Expenses       797.62       466.36       326.69   
   11    

----

In [2]:
import pandas as pd
import openpyxl

In [3]:
workbook = openpyxl.Workbook()
worksheet = workbook.create_sheet("summary", 0)

worksheet.cell(1, 1, "Análiis de la Empresa")

# score_final
worksheet.cell(2, 1, "Calificación Financiero Global")
worksheet.cell(2, 1 + 1, response["financials"]["score_final"])

# score income
worksheet.cell(3, 1, "Calificación Estado de Resultados")
worksheet.cell(3, 1 + 1, response["financials"]["income"]["score_final"])
# score balance
worksheet.cell(4, 1, "Calificación Balance General")
worksheet.cell(4, 1 + 1, response["financials"]["balance"]["score_final"])
# score cash_flow
worksheet.cell(5, 1, "Calificación Flujo de Caja")
worksheet.cell(5, 1 + 1, response["financials"]["cash_flow"]["score_final"])


# score precio historico
worksheet.cell(7, 1, "Calificación Precio Historico")
worksheet.cell(7, 1 + 1, response["price_historic"]["score_final"])

# score precio historico
worksheet.cell(9, 1, "Calificación Precio vs Competencia")
worksheet.cell(9, 1 + 1, str(response["price_competitors"]["score_final"].to_dict()).replace(", ", "\n").replace("'", "").replace("{", "").replace("}", ""))


# # ==================
worksheet = workbook.create_sheet("detalle_estado_resultados", 1)
worksheet.cell(1, 1, "Estado de Resultados de la Empresa")

worksheet.cell(2, 1, "¿La empresa viene creciendo?")

worksheet.cell(3, 1, "Score de Crecimiento de la Empresa")
worksheet.cell(3, 1 + 1, response["financials"]["income"]["score_stmt_growth"])

worksheet.cell(4, 1, "Detalle:")
workbook.save('resultados.xlsx')

# create a writer with parameters mode and if_sheet_exists
writer = pd.ExcelWriter(
    path="resultados.xlsx",
    engine='openpyxl',
    mode='a',                   # append data to the file
    if_sheet_exists='overlay'   # keep previous data on existing sheets
)

# # update data and write changes to the file
df_inc = response["financials"]["income"]["income"].T
df_inc.to_excel(
    writer,                    # use openpyxl writer
    sheet_name='detalle_estado_resultados',
    header=True,              # with no headers
    index=True,               # and no index printed
    startrow=5,                # at this specific position 
    startcol=0,                # with rows and cols numbering from 0
)
# update data and write changes to the file
df_inc_growth = response["financials"]["income"]["detail_growth"].T
df_inc_growth.to_excel(
    writer,                    # use openpyxl writer
    sheet_name='detalle_estado_resultados',
    header=True,              # with no headers
    index=True,               # and no index printed
    startrow=5+df_inc.shape[0]+2,                # at this specific position 
    startcol=0,                # with rows and cols numbering from 0
)
writer.close()

# --
workbook = openpyxl.load_workbook('resultados.xlsx')
worksheet = workbook["detalle_estado_resultados"]

newix = 5+df_inc.shape[0]+df_inc_growth.shape[0]+5
worksheet.cell(newix, 1, "¿Cómo están los margenes vs la competencia?")

worksheet.cell(newix+1, 1, "Score de margenes vs la competencia")
worksheet.cell(newix+1, 1 + 1, response["financials"]["income"]["score_margins_peers"])

worksheet.cell(newix+2, 1, "Margenes de la compañía")
worksheet.cell(newix+2, 1 + 1, str(response["financials"]["income"]["margin_ticker_interes"]).replace(", ", "\n").replace("'", "").replace("{", "").replace("}", ""))

workbook.save('resultados.xlsx')


writer = pd.ExcelWriter(
    path="resultados.xlsx",
    engine='openpyxl',
    mode='a',                   # append data to the file
    if_sheet_exists='overlay'   # keep previous data on existing sheets
)
df_margins = pd.DataFrame(response["financials"]["income"]["margin_peers"])
df_margins.to_excel(
    writer,                    # use openpyxl writer
    sheet_name='detalle_estado_resultados',
    header=True,              # with no headers
    index=True,               # and no index printed
    startrow=newix+3,                # at this specific position 
    startcol=0,                # with rows and cols numbering from 0
)

writer.close()

In [4]:
workbook = openpyxl.load_workbook('resultados.xlsx')
worksheet = workbook.create_sheet("detalle_balance", 2)

worksheet.cell(1, 1, "Gastos operativos mensuales")
worksheet.cell(1, 1+1, response["financials"]["balance"]["balance"]["operation_expenses_month"])

worksheet.cell(2, 1, "Cash")
worksheet.cell(2, 1+1, response["financials"]["balance"]["balance"]["cash"])

worksheet.cell(3, 1, "Meses de operación")
worksheet.cell(3, 1+1, response["financials"]["balance"]["balance"]["months_operation"])

worksheet.cell(4, 1, "Current Ratio")
worksheet.cell(4, 1+1, response["financials"]["balance"]["balance"]["current_ratio"])

worksheet.cell(5, 1, "Quick Ratio")
worksheet.cell(5, 1+1, response["financials"]["balance"]["balance"]["quick_ratio"])

worksheet.cell(6, 1, "Debt Ratio")
worksheet.cell(6, 1+1, response["financials"]["balance"]["balance"]["debt_ratio"])

workbook.save('resultados.xlsx')

In [5]:
writer = pd.ExcelWriter(
    path="resultados.xlsx",
    engine='openpyxl',
    mode='a',                   # append data to the file
    if_sheet_exists='overlay'   # keep previous data on existing sheets
)
df_cash_flow = response["financials"]["cash_flow"]["cash_flow"].T
df_cash_flow.to_excel(
    writer,                    # use openpyxl writer
    sheet_name='detalle_flujo_caja',
    header=True,              # with no headers
    index=True,               # and no index printed
    startrow=0,                # at this specific position 
    startcol=0,                # with rows and cols numbering from 0
)

df_cash_growth = response["financials"]["cash_flow"]["detail_growth"].T
df_cash_growth.to_excel(
    writer,                    # use openpyxl writer
    sheet_name='detalle_flujo_caja',
    header=True,              # with no headers
    index=True,               # and no index printed
    startrow=df_cash_flow.shape[0]+2,                # at this specific position 
    startcol=0,                # with rows and cols numbering from 0
)

df_multip = pd.DataFrame(response["price_historic"]["detail_multiples"])
df_multip.to_excel(
    writer,                    # use openpyxl writer
    sheet_name='detalle_precio',
    header=True,              # with no headers
    index=True,               # and no index printed
    startrow=0,                # at this specific position 
    startcol=0,                # with rows and cols numbering from 0
)

df_multip_peers = pd.DataFrame(response["price_competitors"]["detail_multiples"]).T
df_multip_peers.to_excel(
    writer,                    # use openpyxl writer
    sheet_name='detalle_precio',
    header=True,              # with no headers
    index=True,               # and no index printed
    startrow=df_multip.shape[0]+2,                # at this specific position 
    startcol=0,                # with rows and cols numbering from 0
)

writer.close()

In [4]:
response.keys()

dict_keys(['financials', 'price_historic', 'price_competitors', 'peers'])

In [43]:
response["price_competitors"].keys()

dict_keys(['detail_multiples', 'score_final'])